Run the statement that will create a new external schema  referencing the Glue Data Catalog database ext_s3.

Now that we have created and external schema to reference the Glue Data Catalog within Amazon Redshift we are ready to query the data that is sitting in Amazon S3.
Query the external table, customer, to use Amazon Redshift Spectrum to query data residing in Amazon S3 using regular SQL

Example query 1: Analyse customer state counts

Example query 2: Add additional filter on birth date

Query 3: We can also use Redshift Spectrum to load data from the S3 datalake and convert it into a Redshift internal table. In this example, we can chose to only load non-PII customer columns into Redshift.

After loading the consignment training data, verify that training data has loaded by running a query against the new table.

Now that your training dataset has been loaded into a Redshift table, you will use it to create a machine learning (ML) model using Redshift ML.

We can start the training process for an Amazon Redshift ML model using the CREATE MODEL  statement. The training dataset is coming from our datalake and is specified in the FROM clause, and TARGET defines which column the model is trying to predict. The FUNCTION clause defines the name of the function that is used to make predictions.

You can check the status of the training job, along with other model information, using the SHOW MODEL  statement.

Run the SHOW MODEL statement against the model that already exist in the account.

Create an external schema to establish connection between Amazon Redshift and Amazon Kinesis Data Stream.

Redshift Streaming reads data from the data stream using materlialized views . The data in Kinesis data stream is in JSON format and this can be ingested as-is into Redshift using the SUPER  data type. Run the below to create a materialized view to ingest streaming data into Redshift.

Now the materialized view has been created, we need to run a REFRESH command. This is where the actual data ingestion happens. Data gets loaded from the Kinesis Data Stream into Amazon Redshift without having to stage it first in Amazon S3. This achieves faster performance and improved latency compared to staging the data first.

Once the refresh has completed you can query the streaming data.

Now the data is available in your Redshift Serverless endpoint, you can start running queries against your streaming data and unify it with other datasets.

For example, we would like to know the distribution of our consignments across different states. This can be done by combining the data held in our consignment_stream materialised view with a Redshift table called fleet_summary. To unpack the contents of the JSON payload held in the consignment_stream materialised view you can use PartiQL  syntax.
Run a query to identify consignment information per state.

You can query the most recent transactions that have been ingested into Redshift using this select statement. It compares the current_timestamp with the order_timestamp to measure ingestion latency. Wait for a few seconds and rerun the same query in a different cell. Notice that the data has changed due to the near realtime capability of Redshift.

The next step is to transform and enrich the streaming data using Redshift SQL to generate additional features that will be used by Redshift ML for its predictions. We will use date and time functions  to identify the day of the week, and calculate the number of days between the order date and target delivery date.

We will also use geospatial functions , specifically ST_DistanceSphere , to calculate the distance between origin and destination locations. The GEOMETRY  data type within Redshift provides a cost-effective way to analyze geospatial data such as longitude and latitudes at scale. In this example, the addresses have already been converted to longitude and latitude. However, if you need to perform geocoding, you can integrate Amazon Location Services with Amazon Redshift  using user-defined functions UDFs. On top of geocoding, the Amazon Location Service also allows you to more accurately calculate route distance  between origin and destination and even specify waypoints along the way.

We are going to use another materialized view to persist these transformations. A materialized view provides a simple yet efficient way to create data pipelines using its incremental refresh capability . Amazon Redshift identifies the incremental changes from the last refresh and only updates the target materialized view based on these changes. In this materialized view, all of our transformations are deterministic so we expect our data to be consistent when going through a full refresh or an incremental refresh.

We can use this enriched data to make predictions on the delay probability of a consignment. Redshift ML  is a feature of Amazon Redshift that allows you to use the power of Amazon Redshift to build, train, and deploy machine learning (ML) models directly within your data warehouse.

You can check the status of the training job by running the SHOW MODEL  statement.

As soon as the model is in a READY state, we can start making predictions on new data that is being streamed into Redshift. Predictions are generated using the Redshift ML function that was defined during the training process. We pass the calculated features from the transformed materialized view into this function and the prediction results will populate the delay_probability column.

This final output is persisted into the consignment_predictions table and AWS Step Functions  is orchestrating the ongoing incremental data load into this target table. We use a table for the final output, instead of a materialized view, because ML predictions has randomness involved and it may give us non-deterministic results. Using a table gives us more control on how data is loaded.

Now that this table has been created, you can query it to see what the Redshift ML model is predicting for the streaming data.